# 3webアプリ作成

目次：https://docs.google.com/document/d/1eDw9PK5Ft0vImu81e7oERB_q-dZvxbFe8bGetZBgEEg/edit

## 3.3 応用実装

### 3.3.1 応用アプリ実装の概要：５つのアプリを作る

### 3.3.2 応用アプリ実装

#### ②株価可視化アプリ

>アプリの概要

<img src="pic/img040.png">  

デプロイ先：https://kabu-step2-ver6-6ijpoom5ze3.streamlit.app/

>動作を構成してる部品の紹介（APIやライブラリ）
API
- yahoo finance
    - 目的：株価情報をyf.Ticker("銘柄番号")の１行で取得できる
    - 参考：https://pypi.org/project/yfinance/

ライブラリ
- streamlit
    - 目的：株価情報を表示するためのフロントエンドを生成する
- pandas
    - 目的：株価APIで得られたデータを扱いやすくするために使う
- yfinance
    - 目的：yahoo financeの株価情報取得手段をライブラリとして提供してくれている。
- altair
    - 目的：pandsで構造化データになった株価情報のデータ可視化のために使う



>ライブラリのインストール

ライブラリ
- streamlit
    - pip install streamlit
- pandas
    - pip install pandas
- yfinance
    - pip install yfinance
- altair
    - pip install altair

>ライブラリの公式ドキュメントの見方や使い方を説明


ライブラリ

- streamlit
    - インストール方法はget startページで確認
    - 参考：https://docs.streamlit.io/library/get-started/installation
    - st.titleなどはコンポーネントページから確認出来る
    - 参考：https://streamlit.io/components
- pandas
    - pd.DataFrame()で使います。
    - 参考：https://pandas.pydata.org/docs/
- yfinance
    - yf.Ticker("銘柄番号")でその銘柄の株価を取得できます。
    - 東証などのシンボルはhttps://support.yahoo-net.jp/PccFinance/s/article/H000006603で検索できる
    - 参考：https://pypi.org/project/yfinance/
- altair
    - alt.Chartでチャート生成
    - オプションは.encode()の中に設定していきます。
    - 参考：https://pypi.org/project/altair/

>それぞれの機能をjupyterで動く最小限を作る

以下がjuptyterにて動く最低限のアプリの流れです。

1. 作るアプリの全体のコードを把握
1. それぞれ機能を解説
    1. 株価取得
    1. チャートによる可視化

1.全体のコード

使うアプリの全体像は下記のようになります。  
これはweb上では動きませんが、手元では動作するアプリです。

以下のコードの動作の流れ
1. 拡張機能であるライブラリをインポート
1. 株価データを要求するための準備
    1. 株価を取得する会社すべてをtickersに代入
    1. 要求する日付区間をdaysに代入
1. 株価データ取得
    1. tickersを元にyf.Ticker("ここ銘柄")で１つ１つデータ取得
    1. dfにデータを格納
    1. dfにtickersにあるすべての企業の株価が格納されている
1. 株価データ整理
    1. 欲しい銘柄だけをcompaniesに代入
    1. companiesを使って取得した情報の中から情報を抽出
1. チャートで可視化
    1. チャートの表示範囲の値をyminとymaxに代入
    1. alt.Chartを使って抽出した株価データを可視化

In [47]:
import streamlit as st # フロントエンドを扱うstreamlitの機能をインポート
import pandas as pd # データフレームを扱う機能をインポート
import yfinance as yf # yahoo financeから株価情報を取得するための機能をインポート
import altair as alt # チャート可視化機能をインポート

#取得する銘柄の名前とキーを変換する一覧を設定
#東証などのシンボルはhttps://support.yahoo-net.jp/PccFinance/s/article/H000006603で検索できる
tickers = {
    'apple': 'AAPL',
    'facebook': 'META',
    'google': 'GOOGL',
    'microsoft': 'MSFT',
    'netflix': 'NFLX',
    'amazon': 'AMZN',
    'TOTO': '5332.T',
    'TOYOTA': '7203.T',
}

print('株価可視化アプリ')
print("こちらは株価可視化ツールです。以下のオプションから表示日数を指定できます。")# サイドバーに表示
print("表示日数選択") # サイドバーに表示

days = 10 # 取得するための日数daysに代入

print(f"過去 {days}日間 の株価") # 取得する日数を表示


def get_data(days, tickers):
    df = pd.DataFrame() # 株価を代入するための空箱を用意

    # 選択した株価の数だけ yf.Tickerでリクエストしてデータを取得する
    for company in tickers.keys():
        tkr = yf.Ticker(tickers[company]) # 設定した銘柄一覧でリクエストの為の7203.Tなどに変換をして、それをyf.Tickerで株価リクエスト
        hist = tkr.history(period=f'{days}d') # スライドバーで指定した日数で取得した情報を絞る
        hist.index = hist.index.strftime('%d %B %Y') # indexを日付のフォーマットに変更
        hist = hist[['Close']] # データを終値だけ抽出
        hist.columns = [company] # データのカラムをyf.Tickerのリクエストした企業名に設定
        hist = hist.T # 欲しい情報が逆なので、転置する
        hist.index.name = 'Name' # indexの名前をNameにする
        df = pd.concat([df, hist]) # 用意した空のデータフレームに設定したhistのデータを結合する
    return df # 返り値としてdfを返す

df = get_data(days, tickers) # リクエストする企業一覧すべてと変換するtickersを引数に株価取得





# 企業名の配列を生成し、companiesに代入
companies = ['google', 'apple','TOYOTA']

data = df.loc[companies] # 取得したデータから抽出するための配列で絞ってdataに代入
print("株価 ", data.sort_index()) # dataにあるindexを表示
data = data.T.reset_index() # dataを抽出して転置

# 企業ごとの別々のカラムにデータを表示する必要ないので企業を１つのカラムに統一
data = pd.melt(data, id_vars=['Date']).rename(
    columns={'value': 'Stock Prices'}
)


# チャートに表示する範囲、それぞれをymin, ymaxに代入
print("株価の範囲指定") # サイドバーに表示
ymin = 0
ymax = 3000

# dataとスライドバーで設定した最大最小値を元にalt.Chartを使って株価チャートを作成
chart = (
    alt.Chart(data)
    .mark_line(opacity=0.8, clip=True)
    .encode(
        x="Date:T",
        y=alt.Y("Stock Prices:Q", stack=None, scale=alt.Scale(domain=[ymin, ymax])),
        color='Name:N'
    )
)

# 作成したチャートをstreamlitで表示
display(chart)

株価可視化アプリ
こちらは株価可視化ツールです。以下のオプションから表示日数を指定できます。
表示日数選択
過去 10日間 の株価
株価  Date    15 May 2023  16 May 2023  17 May 2023  18 May 2023  19 May 2023  \
Name                                                                      
TOYOTA          NaN          NaN  1928.000000  1955.500000  1959.500000   
apple    172.070007   172.070007   172.690002   175.050003   175.160004   
google   116.510002   119.510002   120.839996   122.830002   122.760002   

Date    22 May 2023  23 May 2023  24 May 2023  25 May 2023  26 May 2023  \
Name                                                                      
TOYOTA  1950.000000  1857.000000  1956.500000  1938.500000  1937.000000   
apple    174.199997   171.559998   171.839996   172.990005   175.429993   
google   125.050003   122.559998   120.900002   123.480003   124.610001   

Date    29 May 2023  30 May 2023  
Name                              
TOYOTA       1928.5       1940.0  
apple           NaN          NaN  
google          NaN          NaN  
株価の

alt.Chart(...)

2.それぞれ機能を解説

2.1株価取得

具体例として、TOYOTAを参考に株価を取得します。  
必要な情報は銘柄：7203.T だけです。

yf.Ticker(ここ銘柄)で情報を取得できます。  
可視化はtkr.history(period='ここ欲しい区間') で構造化データを取得できます。

In [60]:
tkr = yf.Ticker("7203.T")
hist = tkr.history(period='10d') 
display(hist)

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2023-05-17,1925.0,1935.5,1921.0,1928.0,16295600,0,0
2023-05-18,1945.0,1958.0,1942.0,1955.5,28317600,0,0
2023-05-19,1963.5,1973.5,1950.5,1959.5,23600700,0,0
2023-05-22,1945.0,1950.0,1934.5,1950.0,25566800,0,0
2023-05-23,1970.0,1979.0,1857.0,1857.0,39793700,0,0
2023-05-24,1949.5,1959.5,1924.0,1956.5,35899300,0,0
2023-05-25,1953.5,1960.0,1937.0,1938.5,26877800,0,0
2023-05-26,1940.5,1956.0,1931.0,1937.0,20230300,0,0
2023-05-29,1959.0,1960.0,1922.0,1928.5,28467700,0,0


数字だけでは、どの企業のデータかわからないので、カラムにTOYOTAの企業名を入れ、終値だけ取得します。

In [61]:
hist.index = hist.index.strftime('%d %B %Y') # indexを日付のフォーマットに変更
hist = hist[['Close']] # データを終値だけ抽出
hist.columns = ["TOYOTA"] # データのカラムをyf.Tickerのリクエストした企業名に設定
display(hist)

,TOYOTA
Date,
17 May 2023,1928.0
18 May 2023,1955.5
19 May 2023,1959.5
22 May 2023,1950.0
23 May 2023,1857.0
24 May 2023,1956.5
25 May 2023,1938.5
26 May 2023,1937.0
29 May 2023,1928.5


注意点として、複数の株価データが日数が違うデータもindexであるDateに順番に格納されてしまうので、  
対策として日付をカラムにします。

.Tで転置ができます。

In [62]:
hist = hist.T # 欲しい情報が逆なので、転置する
hist.index.name = 'Name' # indexの名前をNameにする
display(hist)

Date,17 May 2023,18 May 2023,19 May 2023,22 May 2023,23 May 2023,24 May 2023,25 May 2023,26 May 2023,29 May 2023,30 May 2023
Name,,,,,,,,,,
TOYOTA,1928.0,1955.5,1959.5,1950.0,1857.0,1956.5,1938.5,1937.0,1928.5,1940.0


これで株価データの取得が完了しました。

2.2チャートによる可視化

チャートにするために、構造化データを日付をx軸、株価をy軸に変更します。
複数のデータがあった方がわかりやすいので、チャートのための構造化データを以下のコードで作成します。

In [72]:
tickers = {
    'apple': 'AAPL',
    'facebook': 'META',
    'google': 'GOOGL',
    'microsoft': 'MSFT',
    'netflix': 'NFLX',
    'amazon': 'AMZN',
    'TOTO': '5332.T',
    'TOYOTA': '7203.T',
}

days = 10

df = pd.DataFrame() # 株価を代入するための空箱を用意

for company in tickers.keys():
    tkr = yf.Ticker(tickers[company]) # 設定した銘柄一覧でリクエストの為の7203.Tなどに変換をして、それをyf.Tickerで株価リクエスト
    hist = tkr.history(period=f'{days}d') # スライドバーで指定した日数で取得した情報を絞る
    hist.index = hist.index.strftime('%d %B %Y') # indexを日付のフォーマットに変更
    hist = hist[['Close']] # データを終値だけ抽出
    hist.columns = [company] # データのカラムをyf.Tickerのリクエストした企業名に設定
    hist = hist.T # 欲しい情報が逆なので、転置する
    hist.index.name = 'Name' # indexの名前をNameにする
    df = pd.concat([df, hist]) # 用意した空のデータフレームに設定したhistのデータを結合する

display(df)

Date,15 May 2023,16 May 2023,17 May 2023,18 May 2023,19 May 2023,22 May 2023,23 May 2023,24 May 2023,25 May 2023,26 May 2023,29 May 2023,30 May 2023
Name,,,,,,,,,,,,
apple,172.070007,172.070007,172.690002,175.050003,175.160004,174.199997,171.559998,171.839996,172.990005,175.429993,NaN,NaN
facebook,238.860001,238.820007,242.490005,246.850006,245.639999,248.320007,246.740005,249.210007,252.690002,262.040009,NaN,NaN
google,116.510002,119.510002,120.839996,122.830002,122.760002,125.050003,122.559998,120.900002,123.480003,124.610001,NaN,NaN
microsoft,308.784973,311.059998,314.000000,318.519989,318.339996,321.179993,315.260010,313.850006,325.920013,332.890015,NaN,NaN
netflix,335.890015,333.750000,339.959991,371.290009,365.359985,363.010010,355.989990,364.850006,359.000000,378.880005,NaN,NaN
amazon,111.199997,113.400002,115.500000,118.150002,116.250000,115.010002,114.989998,116.750000,115.000000,120.110001,NaN,NaN
TOTO,NaN,NaN,4625.000000,4595.000000,4550.000000,4600.000000,4475.000000,4335.000000,4280.000000,4295.000000,4285.0,4225.0
TOYOTA,NaN,NaN,1928.000000,1955.500000,1959.500000,1950.000000,1857.000000,1956.500000,1938.500000,1937.000000,1928.5,1940.0


構造化データを日付をx軸、株価をy軸するために転置します。

In [73]:
data = df # 全体像のコードと一致させるために変数名を変更します。
data = data.T.reset_index() # dataを抽出して転置
display(data)

Name,Date,apple,facebook,google,microsoft,netflix,amazon,TOTO,TOYOTA
0,15 May 2023,172.070007,238.860001,116.510002,308.784973,335.890015,111.199997,NaN,NaN
1,16 May 2023,172.070007,238.820007,119.510002,311.059998,333.750000,113.400002,NaN,NaN
2,17 May 2023,172.690002,242.490005,120.839996,314.000000,339.959991,115.500000,4625.0,1928.0
3,18 May 2023,175.050003,246.850006,122.830002,318.519989,371.290009,118.150002,4595.0,1955.5
4,19 May 2023,175.160004,245.639999,122.760002,318.339996,365.359985,116.250000,4550.0,1959.5
5,22 May 2023,174.199997,248.320007,125.050003,321.179993,363.010010,115.010002,4600.0,1950.0
6,23 May 2023,171.559998,246.740005,122.559998,315.260010,355.989990,114.989998,4475.0,1857.0
7,24 May 2023,171.839996,249.210007,120.900002,313.850006,364.850006,116.750000,4335.0,1956.5
8,25 May 2023,172.990005,252.690002,123.480003,325.920013,359.000000,115.000000,4280.0,1938.5
9,26 May 2023,175.429993,262.040009,124.610001,332.890015,378.880005,120.110001,4295.0,1937.0


バラバラのデータである必要がないので、企業を１つのがカラムにまとめます。  
使い方は以下のようになります。

pd.melt(ここにデータ, id_vars=[変更した良くない列]).rename(
    columns={'value': '新しいカラムの名前'}
)

In [74]:
data = pd.melt(data, id_vars=['Date']).rename(
    columns={'value': 'Stock Prices'}
)
display(data)

,Date,Name,Stock Prices
0,15 May 2023,apple,172.070007
1,16 May 2023,apple,172.070007
2,17 May 2023,apple,172.690002
3,18 May 2023,apple,175.050003
4,19 May 2023,apple,175.160004
...,...,...,...
91,24 May 2023,TOYOTA,1956.500000
92,25 May 2023,TOYOTA,1938.500000
93,26 May 2023,TOYOTA,1937.000000
94,29 May 2023,TOYOTA,1928.500000


2.2チャートによる可視化

alt.Chartを使って、チャートデータを作成します。  
指定している設定は以下のようになります。

- チャートに使うデータをdata
- xのラベルにDate　色種類は:T
- yのラベルにStock Prices　色種類は:Q　範囲0, 3000
- 全体の色種類指定:N

In [76]:
chart = (
    alt.Chart(data)
    .mark_line(opacity=0.8, clip=True)
    .encode(
        x="Date:T",
        y=alt.Y("Stock Prices:Q", stack=None, scale=alt.Scale(domain=[0, 3000])),
        color='Name:N'
    )
)

# 作成したチャートをstreamlitで表示
display(chart)


alt.Chart(...)

以上がアプリの動作部分です。

>streamlitに実装する場合の使う機能

今はユーザーがブラウザ上で選択したりなど、想定していませんが、  
選択肢の表示などのコンポーネントを紹介します。

※streamlitはjupyterでは動作せず、実行ファイルでのみ動作します

タイトルは  
st.title("ここにタイトル")

ただの表示は  
st.write("ここに表示したい文字")

ただし、再読み込みしない限り、この表示された文字は変更されません。

選択肢は  st.selectbox("ここに表示したい項目名",ここに配列のデータ)  
でできます。

サイドバーに配置したい場合は、   
st.slider.write("ここに表示したい文字")のように.sliderをstのうしろにつけます。

スライドバーの表示は  
st.sidebar.slider('ここに表示したい文字', 最小値の数字, 最大値の数字, 初期の位置の値)  
でできます。

チャートの表示は  
st.altair_chart(ここにちゃーとのデータ, use_container_width=True)でできます。

以上の機能を使ってwebアプリにしていきます。

>（演習）streamlitに実装しましょう！

手元で動いたアプリをstreamlitで動くようにします。  
この演習では必ず実行ファイルにコーディングしてください。

※streamlit run ファイル名で実行する時は必ず実行ファイルで実行してください。

実装の流れ
1. 取得した全株価から抽出する条件companiesの内容をセレクターで取得
    1. 構造化データから選択肢抽出
    1. companiesにセレクトボックスの値が入るようにする
1. 固定数字をスライドバーで変数へ
1. ターミナルに出力していた文字などをブラウザに出力

以下の手元で動いたコードをstreamlitで動くようにします。

In [ ]:
import streamlit as st # フロントエンドを扱うstreamlitの機能をインポート
import pandas as pd # データフレームを扱う機能をインポート
import yfinance as yf # yahoo financeから株価情報を取得するための機能をインポート
import altair as alt # チャート可視化機能をインポート

#取得する銘柄の名前とキーを変換する一覧を設定
#東証などのシンボルはhttps://support.yahoo-net.jp/PccFinance/s/article/H000006603で検索できる
tickers = {
    'apple': 'AAPL',
    'facebook': 'META',
    'google': 'GOOGL',
    'microsoft': 'MSFT',
    'netflix': 'NFLX',
    'amazon': 'AMZN',
    'TOTO': '5332.T',
    'TOYOTA': '7203.T',
}

print('株価可視化アプリ')
print("こちらは株価可視化ツールです。以下のオプションから表示日数を指定できます。")# サイドバーに表示
print("表示日数選択") # サイドバーに表示

days = 10 # 取得するための日数daysに代入

print(f"過去 {days}日間 の株価") # 取得する日数を表示


def get_data(days, tickers):
    df = pd.DataFrame() # 株価を代入するための空箱を用意

    # 選択した株価の数だけ yf.Tickerでリクエストしてデータを取得する
    for company in tickers.keys():
        tkr = yf.Ticker(tickers[company]) # 設定した銘柄一覧でリクエストの為の7203.Tなどに変換をして、それをyf.Tickerで株価リクエスト
        hist = tkr.history(period=f'{days}d') # スライドバーで指定した日数で取得した情報を絞る
        hist.index = hist.index.strftime('%d %B %Y') # indexを日付のフォーマットに変更
        hist = hist[['Close']] # データを終値だけ抽出
        hist.columns = [company] # データのカラムをyf.Tickerのリクエストした企業名に設定
        hist = hist.T # 欲しい情報が逆なので、転置する
        hist.index.name = 'Name' # indexの名前をNameにする
        df = pd.concat([df, hist]) # 用意した空のデータフレームに設定したhistのデータを結合する
    return df # 返り値としてdfを返す

df = get_data(days, tickers) # リクエストする企業一覧すべてと変換するtickersを引数に株価取得





# 企業名の配列を生成し、companiesに代入
companies = ['google', 'apple','TOYOTA']

data = df.loc[companies] # 取得したデータから抽出するための配列で絞ってdataに代入
print("株価 ", data.sort_index()) # dataにあるindexを表示
data = data.T.reset_index() # dataを抽出して転置

# 企業ごとの別々のカラムにデータを表示する必要ないので企業を１つのカラムに統一
data = pd.melt(data, id_vars=['Date']).rename(
    columns={'value': 'Stock Prices'}
)


# チャートに表示する範囲、それぞれをymin, ymaxに代入
print("株価の範囲指定") # サイドバーに表示
ymin = 0
ymax = 3000

# dataとスライドバーで設定した最大最小値を元にalt.Chartを使って株価チャートを作成
chart = (
    alt.Chart(data)
    .mark_line(opacity=0.8, clip=True)
    .encode(
        x="Date:T",
        y=alt.Y("Stock Prices:Q", stack=None, scale=alt.Scale(domain=[ymin, ymax])),
        color='Name:N'
    )
)

# 作成したチャートをstreamlitで表示
display(chart)

1.取得した全株価から抽出する条件companiesの内容をセレクターで取得

df.indexで構造化データから取得した企業のリストを取得できるので、これをlist関数を使って配列にする。

In [78]:
list(df.index)

['apple',
 'facebook',
 'google',
 'microsoft',
 'netflix',
 'amazon',
 'TOTO',
 'TOYOTA']

セレクトボックスの値をcompaniesに代入する。

使い方は  
st.selectbox("ここに表示したい項目名",ここに配列のデータ,初期で選択されている配列)

初期で選択されている配列はどれでも構いませんが、ここでは['google', 'apple','TOYOTA']を指定しておきます。

In [ ]:
companies = st.multiselect(
    '会社名を選択してください。',
    list(df.index),
    ['google', 'apple','TOYOTA'],
)

2.固定数字をスライドバーで変数へ

10日で固定しているdaysとチャートの表示範囲ymin,ymaxをスライドバーから取得します。  
変更する固定数字は以下の３つです。

In [ ]:
days = 10

ymin = 0
ymax = 3000

スライドバーの使い方は以下のようになります。  
st.slider('ここに表示したい文字', 最小値の数字, 最大値の数字, 初期の位置の値) 

また１つのスライドバーから２つの値を取得する場合は、

小さい方の数字, 大きいの方の数字 = st.sidebar.slider(
    'ここに表示したい文字',
    最小値の数字, 最大値の数字, (最小値の数字, 最大値の数字)
)

でできます。

In [ ]:
days = st.sidebar.slider('日数', 1, 50, 20)

ymin, ymax = st.slider(
    '範囲を指定してください。',
    0.0, 5000.0, (0.0, 5000.0)
)

streamlitに実装したアプリをまとめると、以下のようになります。

In [ ]:
import streamlit as st # フロントエンドを扱うstreamlitの機能をインポート
import pandas as pd # データフレームを扱う機能をインポート
import yfinance as yf # yahoo financeから株価情報を取得するための機能をインポート
import altair as alt # チャート可視化機能をインポート

#取得する銘柄の名前とキーを変換する一覧を設定
#東証などのシンボルはhttps://support.yahoo-net.jp/PccFinance/s/article/H000006603で検索できる
tickers = {
    'apple': 'AAPL',
    'facebook': 'META',
    'google': 'GOOGL',
    'microsoft': 'MSFT',
    'netflix': 'NFLX',
    'amazon': 'AMZN',
    'TOTO': '5332.T',
    'TOYOTA': '7203.T',
}

print('株価可視化アプリ')
print("こちらは株価可視化ツールです。以下のオプションから表示日数を指定できます。")# サイドバーに表示
print("表示日数選択") # サイドバーに表示

days = st.sidebar.slider('日数', 1, 50, 20) # 取得するための日数daysに代入

print(f"過去 {days}日間 の株価") # 取得する日数を表示


def get_data(days, tickers):
    df = pd.DataFrame() # 株価を代入するための空箱を用意

    # 選択した株価の数だけ yf.Tickerでリクエストしてデータを取得する
    for company in tickers.keys():
        tkr = yf.Ticker(tickers[company]) # 設定した銘柄一覧でリクエストの為の7203.Tなどに変換をして、それをyf.Tickerで株価リクエスト
        hist = tkr.history(period=f'{days}d') # スライドバーで指定した日数で取得した情報を絞る
        hist.index = hist.index.strftime('%d %B %Y') # indexを日付のフォーマットに変更
        hist = hist[['Close']] # データを終値だけ抽出
        hist.columns = [company] # データのカラムをyf.Tickerのリクエストした企業名に設定
        hist = hist.T # 欲しい情報が逆なので、転置する
        hist.index.name = 'Name' # indexの名前をNameにする
        df = pd.concat([df, hist]) # 用意した空のデータフレームに設定したhistのデータを結合する
    return df # 返り値としてdfを返す

df = get_data(days, tickers) # リクエストする企業一覧すべてと変換するtickersを引数に株価取得





# 企業名の配列を生成し、companiesに代入
companies = st.multiselect(
    '会社名を選択してください。',
    list(df.index),
    ['google', 'apple','TOYOTA'],
)

data = df.loc[companies] # 取得したデータから抽出するための配列で絞ってdataに代入
print("株価 ", data.sort_index()) # dataにあるindexを表示
data = data.T.reset_index() # dataを抽出して転置

# 企業ごとの別々のカラムにデータを表示する必要ないので企業を１つのカラムに統一
data = pd.melt(data, id_vars=['Date']).rename(
    columns={'value': 'Stock Prices'}
)


# チャートに表示する範囲、それぞれをymin, ymaxに代入
print("株価の範囲指定") # サイドバーに表示


ymin, ymax = st.slider(
    '範囲を指定してください。',
    0.0, 5000.0, (0.0, 5000.0)
)

# dataとスライドバーで設定した最大最小値を元にalt.Chartを使って株価チャートを作成
chart = (
    alt.Chart(data)
    .mark_line(opacity=0.8, clip=True)
    .encode(
        x="Date:T",
        y=alt.Y("Stock Prices:Q", stack=None, scale=alt.Scale(domain=[ymin, ymax])),
        color='Name:N'
    )
)

# 作成したチャートをstreamlitで表示
display(chart)

3.ターミナルに出力していた文字などをブラウザに出力

最後にprintでターミナルで出力していた  
タイトルや文字やチャートを以下の４つのコードを使ってブラウザに出力します。

- st.title
- st.write
- st.altair_chart

In [ ]:
import streamlit as st # フロントエンドを扱うstreamlitの機能をインポート
import pandas as pd # データフレームを扱う機能をインポート
import yfinance as yf # yahoo financeから株価情報を取得するための機能をインポート
import altair as alt # チャート可視化機能をインポート

#取得する銘柄の名前とキーを変換する一覧を設定
#東証などのシンボルはhttps://support.yahoo-net.jp/PccFinance/s/article/H000006603で検索できる
tickers = {
    'apple': 'AAPL',
    'facebook': 'META',
    'google': 'GOOGL',
    'microsoft': 'MSFT',
    'netflix': 'NFLX',
    'amazon': 'AMZN',
    'TOTO': '5332.T',
    'TOYOTA': '7203.T',
}

st.title('株価可視化アプリ') # タイトル

st.write("こちらは株価可視化ツールです。以下のオプションから表示日数を指定できます。") # サイドバーに表示

st.write("表示日数選択") # サイドバーに表示

days = st.slider('日数', 1, 50, 20) # サイドバーに表示　取得するための日数をスライドバーで表示し、daysに代入

st.write(f"過去 {days}日間 の株価") # 取得する日数を表示

# @st.cache_dataで読み込みが早くなるように処理を保持しておける
@st.cache_data
def get_data(days, tickers):
    df = pd.DataFrame() # 株価を代入するための空箱を用意

    # 選択した株価の数だけ yf.Tickerでリクエストしてデータを取得する
    for company in tickers.keys():
        tkr = yf.Ticker(tickers[company]) # 設定した銘柄一覧でリクエストの為の7203.Tなどに変換をして、それをyf.Tickerで株価リクエスト
        hist = tkr.history(period=f'{days}d') # スライドバーで指定した日数で取得した情報を絞る
        hist.index = hist.index.strftime('%d %B %Y') # indexを日付のフォーマットに変更
        hist = hist[['Close']] # データを終値だけ抽出
        hist.columns = [company] # データのカラムをyf.Tickerのリクエストした企業名に設定
        hist = hist.T # 欲しい情報が逆なので、転置する
        hist.index.name = 'Name' # indexの名前をNameにする
        df = pd.concat([df, hist]) # 用意した空のデータフレームに設定したhistのデータを結合する
    return df # 返り値としてdfを返す

# チャートに表示する範囲をスライドで表示し、それぞれをymin, ymaxに代入
st.write("株価の範囲指定") # サイドバーに表示
ymin, ymax = st.slider(
    '範囲を指定してください。',
    0.0, 5000.0, (0.0, 5000.0)
) # サイドバーに表示

df = get_data(days, tickers) # リクエストする企業一覧すべてと変換するtickersを引数に株価取得


# 取得したデータから抽出するための配列を生成し、companiesに代入
companies = st.multiselect(
    '会社名を選択してください。',
    list(df.index),
    ['google', 'apple','TOYOTA'], # 最初に表示する企業名を設定
)


data = df.loc[companies] # 取得したデータから抽出するための配列で絞ってdataに代入
st.write("株価 ", data.sort_index()) # dataにあるindexを表示
data = data.T.reset_index() # dataを抽出して転置

# 企業ごとの別々のカラムにデータを表示する必要ないので企業を１つのカラムに統一
data = pd.melt(data, id_vars=['Date']).rename(
    columns={'value': 'Stock Prices'}
)

# dataとスライドバーで設定した最大最小値を元にalt.Chartを使って株価チャートを作成
chart = (
    alt.Chart(data)
    .mark_line(opacity=0.8, clip=True)
    .encode(
        x="Date:T",
        y=alt.Y("Stock Prices:Q", stack=None, scale=alt.Scale(domain=[ymin, ymax])),
        color='Name:N'
    )
)

# 作成したチャートをstreamlitで表示
st.altair_chart(chart, use_container_width=True)

最後にサイドバーに表示する項目に.sidebarをつけます。

In [ ]:
import streamlit as st # フロントエンドを扱うstreamlitの機能をインポート
import pandas as pd # データフレームを扱う機能をインポート
import yfinance as yf # yahoo financeから株価情報を取得するための機能をインポート
import altair as alt # チャート可視化機能をインポート

#取得する銘柄の名前とキーを変換する一覧を設定
#東証などのシンボルはhttps://support.yahoo-net.jp/PccFinance/s/article/H000006603で検索できる
tickers = {
    'apple': 'AAPL',
    'facebook': 'META',
    'google': 'GOOGL',
    'microsoft': 'MSFT',
    'netflix': 'NFLX',
    'amazon': 'AMZN',
    'TOTO': '5332.T',
    'TOYOTA': '7203.T',
}

st.title('株価可視化アプリ') # タイトル

st.sidebar.write("こちらは株価可視化ツールです。以下のオプションから表示日数を指定できます。") # サイドバーに表示

st.sidebar.write("表示日数選択") # サイドバーに表示

days = st.sidebar.slider('日数', 1, 50, 20) # サイドバーに表示　取得するための日数をスライドバーで表示し、daysに代入

st.write(f"過去 {days}日間 の株価") # 取得する日数を表示

# @st.cache_dataで読み込みが早くなるように処理を保持しておける
@st.cache_data
def get_data(days, tickers):
    df = pd.DataFrame() # 株価を代入するための空箱を用意

    # 選択した株価の数だけ yf.Tickerでリクエストしてデータを取得する
    for company in tickers.keys():
        tkr = yf.Ticker(tickers[company]) # 設定した銘柄一覧でリクエストの為の7203.Tなどに変換をして、それをyf.Tickerで株価リクエスト
        hist = tkr.history(period=f'{days}d') # スライドバーで指定した日数で取得した情報を絞る
        hist.index = hist.index.strftime('%d %B %Y') # indexを日付のフォーマットに変更
        hist = hist[['Close']] # データを終値だけ抽出
        hist.columns = [company] # データのカラムをyf.Tickerのリクエストした企業名に設定
        hist = hist.T # 欲しい情報が逆なので、転置する
        hist.index.name = 'Name' # indexの名前をNameにする
        df = pd.concat([df, hist]) # 用意した空のデータフレームに設定したhistのデータを結合する
    return df # 返り値としてdfを返す

# チャートに表示する範囲をスライドで表示し、それぞれをymin, ymaxに代入
st.sidebar.write("株価の範囲指定") # サイドバーに表示
ymin, ymax = st.sidebar.slider(
    '範囲を指定してください。',
    0.0, 5000.0, (0.0, 5000.0)
) # サイドバーに表示

df = get_data(days, tickers) # リクエストする企業一覧すべてと変換するtickersを引数に株価取得


# 取得したデータから抽出するための配列を生成し、companiesに代入
companies = st.multiselect(
    '会社名を選択してください。',
    list(df.index),
    ['google', 'apple','TOYOTA'], # 最初に表示する企業名を設定
)


data = df.loc[companies] # 取得したデータから抽出するための配列で絞ってdataに代入
st.write("株価 ", data.sort_index()) # dataにあるindexを表示
data = data.T.reset_index() # dataを抽出して転置

# 企業ごとの別々のカラムにデータを表示する必要ないので企業を１つのカラムに統一
data = pd.melt(data, id_vars=['Date']).rename(
    columns={'value': 'Stock Prices'}
)

# dataとスライドバーで設定した最大最小値を元にalt.Chartを使って株価チャートを作成
chart = (
    alt.Chart(data)
    .mark_line(opacity=0.8, clip=True)
    .encode(
        x="Date:T",
        y=alt.Y("Stock Prices:Q", stack=None, scale=alt.Scale(domain=[ymin, ymax])),
        color='Name:N'
    )
)

# 作成したチャートをstreamlitで表示
st.altair_chart(chart, use_container_width=True)